In [26]:
import sqlite3
import configparser
from sqlalchemy import create_engine, text 
import pandas as pd


In [37]:
from mysql.connector import MySQLConnection, Error

# Making connection to SQL server
def make_connection():
    try:
        conn = MySQLConnection(host='IES-ADS-ClassDB.sjsu.edu',
                               database='querycrew_db',
                               user='querycrew_user',
                               password='Pomegranate_746')
        
        if conn.is_connected():
            print('Connected to the MySQL database!')
            
            return conn
                
    except Error as e:
        print('Connection failed.')
        print(e)
        
        return None

In [38]:
conn = make_connection()
cursor = conn.cursor()

Connected to the MySQL database!


In [43]:
cursor.execute("DROP TABLE IF EXISTS Driver;")
cursor.execute("DROP TABLE IF EXISTS location;")
cursor.execute("DROP TABLE IF EXISTS Vehicle;")
cursor.execute("DROP TABLE IF EXISTS vehicle_inspection")
cursor.execute("DROP TABLE IF EXISTS station;")
cursor.execute("DROP TABLE IF EXISTS county;")
cursor.execute("DROP TABLE IF EXISTS IsLocatedIn;") 

conn.commit()
print("All tables dropped successfully!")

All tables dropped successfully!


In [44]:
# Create the tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS county
(
  county_name INT NOT NULL,
  county_id INT NOT NULL,
  PRIMARY KEY (county_id)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS station
(
  station_id INT NOT NULL,
  station_name INT NOT NULL,
  PRIMARY KEY (station_id)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS vehicle_inspection
(
  inspection_id INT NOT NULL,
  date INT NOT NULL,
  status INT NOT NULL,
  station_id INT NOT NULL,
  PRIMARY KEY (inspection_id),
  FOREIGN KEY (station_id) REFERENCES station(station_id)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Vehicle
(
  model_year INT NOT NULL,
  base_msrp INT,
  vin INT NOT NULL,
  make INT NOT NULL,
  model INT NOT NULL,
  inspection_id INT NOT NULL,
  PRIMARY KEY (vin),
  FOREIGN KEY (inspection_id) REFERENCES vehicle_inspection(inspection_id)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS location
(
  zip_code INT NOT NULL,
  city INT NOT NULL,
  State INT NOT NULL,
  DOL_ID INT NOT NULL,
  vin INT NOT NULL,
  station_id INT NOT NULL,
  county_id INT NOT NULL,
  PRIMARY KEY (DOL_ID),
  FOREIGN KEY (vin) REFERENCES Vehicle(vin),
  FOREIGN KEY (station_id) REFERENCES station(station_id),
  FOREIGN KEY (county_id) REFERENCES county(county_id)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Driver
(
  first INT NOT NULL,
  last INT NOT NULL,
  email INT NOT NULL,
  vin INT NOT NULL,
  PRIMARY KEY (email),
  FOREIGN KEY (vin) REFERENCES Vehicle(vin)
);
''')


In [45]:
# Insert example data into the station table
cursor.execute("INSERT INTO station (station_id, station_name) VALUES (1, 101)")
cursor.execute("INSERT INTO station (station_id, station_name) VALUES (2, 102)")
cursor.execute("INSERT INTO station (station_id, station_name) VALUES (3, 103)")
cursor.execute("INSERT INTO station (station_id, station_name) VALUES (4, 104)")
cursor.execute("INSERT INTO station (station_id, station_name) VALUES (5, 105)")

# Insert example data into the vehicle_inspection table
cursor.execute("INSERT INTO vehicle_inspection (inspection_id, date, status, station_id) VALUES (1, 20230101, 1, 1)")
cursor.execute("INSERT INTO vehicle_inspection (inspection_id, date, status, station_id) VALUES (2, 20230102, 0, 2)")
cursor.execute("INSERT INTO vehicle_inspection (inspection_id, date, status, station_id) VALUES (3, 20230103, 1, 3)")
cursor.execute("INSERT INTO vehicle_inspection (inspection_id, date, status, station_id) VALUES (4, 20230104, 0, 4)")
cursor.execute("INSERT INTO vehicle_inspection (inspection_id, date, status, station_id) VALUES (5, 20230105, 1, 5)")

# Insert example data into the Vehicle table
cursor.execute("INSERT INTO Vehicle (model_year, base_msrp, vin, make, model, inspection_id) VALUES (2020, 30000, 12345, 1, 1, 1)")
cursor.execute("INSERT INTO Vehicle (model_year, base_msrp, vin, make, model, inspection_id) VALUES (2019, 25000, 12346, 2, 2, 2)")
cursor.execute("INSERT INTO Vehicle (model_year, base_msrp, vin, make, model, inspection_id) VALUES (2021, 35000, 12347, 3, 3, 3)")
cursor.execute("INSERT INTO Vehicle (model_year, base_msrp, vin, make, model, inspection_id) VALUES (2018, 20000, 12348, 4, 4, 4)")
cursor.execute("INSERT INTO Vehicle (model_year, base_msrp, vin, make, model, inspection_id) VALUES (2022, 40000, 12349, 5, 5, 5)")

# Add a named foreign key constraint using ALTER TABLE
cursor.execute('''
ALTER TABLE Vehicle
ADD CONSTRAINT fk_inspection
FOREIGN KEY (inspection_id)
REFERENCES vehicle_inspection(inspection_id);
''')

# Commit the changes
conn.commit()


In [46]:
# Query the tables and display the results in dataframes
vehicle_inspection_df = pd.read_sql_query("SELECT * FROM vehicle_inspection", conn)
vehicle_df = pd.read_sql_query("SELECT * FROM Vehicle", conn)

print("Vehicle Inspection Table:")
print(vehicle_inspection_df)

print("\nVehicle Table:")
print(vehicle_df)

# Close the connection
conn.close()

Vehicle Inspection Table:
   inspection_id      date  status  station_id
0              1  20230101       1           1
1              2  20230102       0           2
2              3  20230103       1           3
3              4  20230104       0           4
4              5  20230105       1           5

Vehicle Table:
   model_year  base_msrp    vin  make  model  inspection_id
0        2020      30000  12345     1      1              1
1        2019      25000  12346     2      2              2
2        2021      35000  12347     3      3              3
3        2018      20000  12348     4      4              4
4        2022      40000  12349     5      5              5


/var/folders/19/_djhhd2d4j1drxs9ryt60rtm0000gn/T/ipykernel_65550/3468094508.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vehicle_inspection_df = pd.read_sql_query("SELECT * FROM vehicle_inspection", conn)
/var/folders/19/_djhhd2d4j1drxs9ryt60rtm0000gn/T/ipykernel_65550/3468094508.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vehicle_df = pd.read_sql_query("SELECT * FROM Vehicle", conn)
